In [1]:
#import splitwavepy as sw
from obspy import read
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

import matplotlib.pyplot as plt
import numpy as np

## to do:
## events
## waveforms
## theoretical arrival times
### do splitting
## save Results
## stack and average

## 3. get waveforms, process, filter, detrend, remove response
## find matching station with matching event
## N,E,Z
#station = stationlist[1]
#t = starttime
### call function for eventtime and SKS time 

### check for maximum value in window to identify real SKS Phase
### write out windows of waveforms..
### Plot all three channels with several incidence times
### Make a proper work flow:
### data exists? Any gaps or spikes?

## Parameters used in Walpols Study: 
## SKS epicentral distance between 95 < delta < 145° 
## distance from wikipedia: 60-141°
## SNR ratio >16
## weighted by SNR 
## Butterworth 0.02-0.1 Hz
## arrival time window between -15,-5 to +15,+30 s
## use eigenvalue method from silver and chan and cross correlation from Andow
## if both disagree by 45° --> Null measurements
## randomly sample and inspect for QC
## lambda2/lambda1 <0.025 (postcorrection), the smaller the fraction, the greater the linear particle motion
## SPOL-BAZ ??




# Check available Swiss Stations: Total 247

In [2]:
client = Client("ETH") # ORFEUS

station= '*'
starttime = UTCDateTime("2000-01-01T00:00:00.000")
endtime = UTCDateTime("2018-12-31T00:00:00.000")

inventory = client.get_stations(network="CH", station=station, starttime=starttime, endtime=endtime)

## Total number of 247 available Stations
print(len(inventory[0]))

stationlist = inventory[0]

station = stationlist[10]
print(station)
print(station.code)
print(station.longitude)
print(station.latitude)
print(station.elevation)
print(station.start_date)
print(station.end_date)

248
Station SSCN (Schweizerhalle, Novartis, BL)
	Station Code: SSCN
	Channel Count: None/None (Selected/Total)
	2014-11-21T09:42:24.000000Z - 
	Access: open 
	Latitude: 47.53, Longitude: 7.67, Elevation: 269.0 m
	Available Channels:

SSCN
7.66973
47.53163
269.0
2014-11-21T09:42:24.000000Z
None


In [3]:
import obspy
## 1. get events catalogue
## events after 1999
#cat = obspy.read_events("/home/hein/Dropbox/PhD/code_PhD/qcmt.ndk")
#cat2 = cat.filter("time > 2000-01-01T00:00", "magnitude >= 5.5", "depth <= 10000")
#cat2.write("/home/hein/Dropbox/PhD/code_PhD/qcmt_edit.cmt",format="CMTSOLUTION")


cat = obspy.read_events("/home/hein/Dropbox/PhD/code_PhD/qcmt.ndk")

#cat = cat.filter("time > {0}".format(station.start_date),"time < 2018-12-01T00:00:00")
cat = cat.filter("time > 2000-01-01T00:00","time < 2018-12-01T00:00:00")
print(cat)

/home/hein/miniconda3/envs/py27/lib/python2.7/site-packages/obspy/io/ndk/core.py:192: ObsPyNDKWarning: Skipped last 4 lines. Not a multiple of 5 lines.
  warnings.warn(msg, ObsPyNDKWarning)


32819 Event(s) in Catalog:
2000-01-01T05:24:39.300000Z | +37.150,  +69.720 | 5.11 Mwc
2000-01-01T05:58:26.700000Z | -60.740, +153.400 | 5.97 Mwc
...
2017-12-31T20:59:06.100000Z | -53.040, -118.290 | 5.28 Mwc
2017-12-31T23:48:50.800000Z | +17.850, +146.080 | 4.97 Mwc
To see all events call 'print(CatalogObject.__str__(print_all=True))'


# REDUCE Catalogue to only Strong Events > Mw7, from epicentral distances between 65-145°
#95- ?
## Long_list contains:
### Station code, start_date, lat, lon, event_time, event_lat,event_lon, depth)

In [4]:
from obspy.geodetics import locations2degrees


cat_m7 = cat.filter("magnitude >= 7.0")

print(cat_m7)

counter = 0
long_list = []

for ev in cat_m7:    
    event1 = ev
    ### Event parameters
    orig  = event1.origins

    mag = event1.magnitudes[0]
    
    for station in stationlist:        
        ### check if station has recorded before event 
        if (float(station.start_date)<float(orig[0].time)) and (station.end_date == None):


            dist_deg = locations2degrees(orig[0].latitude, orig[0].longitude,
                           station.latitude, station.longitude)
            ## check wheter distance is in the right range            
            if (dist_deg>95) and (dist_deg<145):
                counter +=1

                tmp = [counter, str(station.code), station.start_date, station.latitude,
                                              station.longitude, orig[0].time,
                                    orig[0].latitude, orig[0].longitude, orig[0].depth, mag.mag, dist_deg]
                long_list.append(tmp)




#np.savetxt("/media/hein/home2/SplitWave_Data/m7_stat_list.csv", long_list, delimiter=', ', header='New Data', comments='# ')
## Save Long List?
### clean up

print(stationlist[0])


print('Station',long_list[0][1])
print('event',long_list[0][5])
print('event',long_list[0][8])
print(len(long_list[:]))


## instert magnitude
## and SNR Quality Criterium



241 Event(s) in Catalog:
2000-01-08T16:47:30.200000Z | -16.840, -173.810 | 7.16 Mwc
2000-02-25T01:44:05.200000Z | -19.550, +174.170 | 7.07 Mwc
...
2017-09-19T18:14:47.200000Z | +18.590,  -98.630 | 7.14 Mwc
2017-11-12T18:18:25.300000Z | +34.830,  +45.840 | 7.4 Mwc
To see all events call 'print(CatalogObject.__str__(print_all=True))'
Station STIEG (Oberembrach, Stiegenhof, ZH)
	Station Code: STIEG
	Channel Count: None/None (Selected/Total)
	2012-12-11T00:00:00.000000Z - 
	Access: open 
	Latitude: 47.50, Longitude: 8.65, Elevation: 637.0 m
	Available Channels:

('Station', 'SRHB')
('event', UTCDateTime(2000, 3, 28, 11, 0, 22, 500000))
('event', 126500.0)
6997


# Skript to automatically download Waveforms from available Earthquakes (done)


In [5]:
#arrivals = model.get_travel_times(source_depth_in_km=100,
#                                  distance_in_degree=100,
#                                  phase_list=["P", 'S',"SKS",'SKKS','PP'])
#arrivals.plot()
#fig, ax = plt.subplots(figsize=(9, 9))
#ax = plot_travel_times(source_depth=10, phase_list=["P", "S", "PP","SKS",'ScS'],
#                       ax=ax, fig=fig, verbose=True)

#plt.plot(arrivals)
#print(arrivals)


In [6]:
import os 

save_loc = '/media/hein/home2/SplitWave_Data/'
FMT = 'SAC'

n_SKS=len(long_list[:])


DO_DOWNLOAD=False
if DO_DOWNLOAD==True:

    for i in range(0,n_SKS):

        try:    
            st = obspy.Stream()
            ## go through EQ time list and download 1h waveforms 
            st = client.get_waveforms("CH", long_list[i][1], "",
                                      "BH?", long_list[i][5],long_list[i][5]+60*60 ,attach_response=True)  
             ## go to folder, save each trace individually        
            path= '{0}/{1}'.format(save_loc,long_list[i][1])

            try:  
                os.mkdir(path)
            except OSError:
                print ("Creation of the directory %s failed" % path)
            else:  
                print ("Successfully created the directory %s " % path)

            pre_filt = (0.001, 0.005, 40.0, 60.0)        
            st.remove_response(pre_filt=pre_filt, plot=False)
            st.decimate(factor=int(st[0].stats.sampling_rate/10), strict_length=False)         ## downsample to 10 Hz
            st.detrend(type='linear')   
            st.filter("bandpass",freqmin=0.01,freqmax=0.5)

            #FORMAT: 2011.164.14.31.26.1450.FR.ASEAF.BHN..SAC
            event_info = UTCDateTime(str(long_list[i][5]))
            for tr in st:
                filename='{0}/{1}/{2}.{3}.{4}.{5}.{6}.{7}.{8}.{9}.{10}..SAC'.format(save_loc,long_list[i][1],event_info.year,
                                                                  event_info.julday,event_info.hour,
                                                                  event_info.minute,event_info.second,
                                                                  event_info.microsecond/100,tr.stats.network,
                                                                  tr.stats.station,tr.stats.channel)        
                tr.write(filename,format=FMT)
                print('saved data for Station: ',long_list[i][1])        
        except:
            print('no data for Station: ',long_list[i][1])
            pass


# First calculates distance from EQ and Station coordinates.

# Then calculates the theoretical arrival Times of each Phases from distance and depth of the event.


In [7]:
import os 
from obspy.taup import TauPyModel
from obspy.geodetics.base import gps2dist_azimuth
from obspy.geodetics import locations2degrees
from obspy.taup import plot_travel_times


def read_station_event_data(istation):
    #### to get all waveforms of 1 station
    st_lat_list = []
    st_lon_list = []        
    ev_lat_list =[]
    ev_lon_list =[]
    ev_time_list = []
    ev_depth_list = []    
    ev_mag_list = []
    ev_dist_list = []
    back_azimut_list =[]    
    t_SKS_list = []
    t_SKKS_list = []
    t_SKS=0     
    t_SKKS=0 
    t_PP=0     
    t_PP_list = []    

    PHASE_LIST = ['PP','SKS','SKKS']
    st_event = obspy.Stream()
    for iSKS in long_list:    
#    for iSKS in long_list[0:25]:
        
        if iSKS[1]==istation:
#            print(iSKS[1])
            filename='{0}/{1}/{2}.{3}.{4}.{5}.{6}.{7}.CH.{8}.BH?..SAC'.format(save_loc,iSKS[1],iSKS[5].year,
                                                              iSKS[5].julday,iSKS[5].hour,
                                                              iSKS[5].minute,iSKS[5].second,
                                                              iSKS[5].microsecond/100,
                                                              iSKS[1])

        ## something which only gets the data for 1 events
#            print(filename)
            st_tmp=obspy.Stream()
            try:
            
        #                print('reading', iSKS[1])
                st_tmp += read(filename)

                if len(st_tmp)>2:

                    if (st_tmp[0].stats.npts > 36000 and st_tmp[1].stats.npts > 36000 and st_tmp[2].stats.npts > 36000):


                        st_lat = iSKS[3]
                        st_lon = iSKS[4]
                        ev_time = UTCDateTime(iSKS[5])     
                        ev_lat = iSKS[6]
                        ev_lon = iSKS[7]
                        ev_depth = iSKS[8]
                        ev_mag = iSKS[9]  
                        ev_dist = iSKS[10]

                        st_lat_list.append(st_lat)
                        st_lon_list.append(st_lon)                    
                        ev_lat_list.append(ev_lat)
                        ev_lon_list.append(ev_lon)        
                        ev_time_list.append(ev_time)
                        ev_depth_list.append(ev_depth)
                        ev_mag_list.append(ev_mag)
                        ev_dist_list.append(ev_dist)

                        dist_deg = locations2degrees(ev_lat,ev_lon,
                                   st_lat,st_lon)

                        arrivals = model.get_travel_times(source_depth_in_km=ev_depth/1000, distance_in_degree=dist_deg,phase_list=PHASE_LIST)

                        geodetics = gps2dist_azimuth(ev_lat, ev_lon,
                               st_lat, st_lon, a=6378137.0, f=0.0033528106647474805)

                        back_azimut = geodetics[2]
                        back_azimut_list.append(back_azimut)

                        for i in range(0,len(arrivals)):
                            if arrivals[i].name=='SKS':
                                t_SKS = arrivals[i].time
                            elif (arrivals[i].name=='SKKS' and t_SKKS==0):
                                t_SKKS = arrivals[i].time     
                            elif arrivals[i].name=='PP':
                                t_PP = arrivals[i].time

                        t_SKS_list.append(t_SKS)
                        t_SKKS_list.append(t_SKKS)                        
                        t_PP_list.append(t_PP)                                                

                        st_event +=st_tmp
                    else:
                        print('Stream has too few samples')
                else:
                    print('Stream has not 3 channels')



            except:
                print('no matching file')
            

    return st_event,st_lat_list,st_lon_list,ev_lat_list,ev_lon_list,ev_time_list,ev_depth_list,ev_mag_list,ev_dist_list,back_azimut_list, t_SKS_list,t_SKKS_list,t_PP_list

### THERE SEEMS TO BE A PROBLEM WITH TAU P NAMING THE PP and S Phase to SKKS
#fig, ax = plt.subplots(figsize=(9, 9))
#ax = plot_travel_times(source_depth=10, phase_list=PHASE_LIST, ax=ax, fig=fig, verbose=True)
#fig.savefig('/media/hein/home2/SplitWave_Results/Project_images/Travel_times_tau-P.png')
#arrivals = model.get_ray_paths(source_depth_in_km=50, distance_in_degree=100, phase_list=PHASE_LIST)

#fig, ax = plt.subplots(figsize=(9, 9))
#fig =plt.figure(figsize=(16,9))
#ax = fig.add_axes([0.2,0.15,0.6,0.7],projection='polar')  
#ax = arrivals.plot_rays(legend=True,fig=fig)
#fig.savefig('/media/hein/home2/SplitWave_Results/Project_images/Ray_path_tau-P.png')

#fig =plt.figure(figsize=(16,9))
#ax = fig.add_axes([0.2,0.15,0.6,0.7])  
#ax = arrivals.plot_rays(legend=True,plot_type="cartesian",fig=fig)
#fig.savefig('/media/hein/home2/SplitWave_Results/Project_images/Ray_path_cartesian_tau-P.png')

In [8]:
### go through for every event

def automatic_SplitWave_Routine(st_ev,st_lat_l,st_lon_l,ev_lat_l,ev_lon_l,ev_time_l,ev_depth_l,ev_mag_l,ev_dist_l,back_azimut_l, t_SKS_l,t_SKKS_l,t_PP_l):
    
    Az_l = []
    SV_Az_l = []
    for step in tqdm(range(0,(len(st_ev)-3)/3)):
#    for step in tqdm(range(2,5)):

        st_selection = st_ev[step*3:3*step+3]

        ### check if event time outside st_event
        back_az = back_azimut_l[step]
        st_cut,SNR,t_real_SKS = plot_SKS_window(st_selection,t_SKS_l[step],t_SKKS_l[step],ev_time_l[step],ev_mag_l[step],ev_dist_l[step])
        
        #### CHEVROT HERE
        Az,SV_Az = SKS_Intensity_Chevrot(st_selection,ev_time_l[step],float(t_real_SKS)-float(ev_time_l[step]),back_azimut_l[step],plot=True)
        Az_l.append(Az)
        SV_Az_l.append(SV_Az)
        ####        
        
        method= 'TransM'
        fast,dfast,lag,dlag =  Splitwave_TransM(st_cut,back_az,plot_SplitW)
        vals = [str(st_cut[0].stats.station),st_lat_l[step],st_lon_l[step],float(ev_time_l[step]),ev_depth_l[step]/1000,ev_mag_l[step],ev_lat_l[step],ev_lon_l[step], fast,dfast,lag,dlag ,SNR]
    #    print(fast,dfast,lag,dlag)                         
        write_SKS_Results(path,method,vals,st_cut[0].stats.station, header)

        method= 'CrossC'
        fast,dfast,lag,dlag =  Splitwave_CrossC(st_cut,plot_SplitW)
        vals = [str(st_cut[0].stats.station),st_lat_l[step],st_lon_l[step],float(ev_time_l[step]),ev_depth_l[step]/1000,ev_mag_l[step],ev_lat_l[step],ev_lon_l[step], fast,dfast,lag,dlag ,SNR]    
    #    print(fast,dfast,lag,dlag)
        write_SKS_Results(path,method,vals,st_cut[0].stats.station, header)

        method= 'Eig3D'
        fast,dfast,lag,dlag =  Splitwave_Eig3D(st_cut,plot_SplitW)
        vals = [str(st_cut[0].stats.station),st_lat_l[step],st_lon_l[step],float(ev_time_l[step]),ev_depth_l[step]/1000,ev_mag_l[step],ev_lat_l[step],ev_lon_l[step], fast,dfast,lag,dlag ,SNR]
    #    print(fast,dfast,lag,dlag)
        write_SKS_Results(path,method,vals,st_cut[0].stats.station, header)    
        
    ### AFTER CALCULATING ALL INTENSITIES FOR ALL EVENTS, GET BEST PARAMS FOR CHEVROT
    dt,phi,std_dt,std_phi = get_best_dt_and_phi(Az_l,SV_Az_l,st_ev[0].stats.station)
    
    print('Splitting intensity analyis: {0}'.format(st_cut[0].stats.station))    
    print('dt',dt)
    print('phi',phi)
    
    return dt,phi,std_dt,std_phi




In [9]:
### easy routine to load in the trace and event parameters
### but also to automatically do it
### set threshold for SNR



def plot_SKS_window(st_event,t_SKS,t_SKKS,ev_time,ev_mag,ev_dist,plot=True):
    
    st_selection = obspy.Stream()
    st_selection = st_event
    twin = 40     ## Time Window 40 s
    t = ev_time
    #### TO MAKE SURE THE TRACES START AND END AT THE SAME TIME AND HAVE THE SAME amount of samples 
    mylist = (float(st_selection[0].stats.starttime),
                           float(st_selection[1].stats.starttime),float(st_selection[2].stats.starttime))
    max_startt = max(mylist)
    id_m = mylist.index(max_startt)
    max_startt = UTCDateTime(max_startt)

    mylist2 = (float(st_selection[0].stats.endtime),
                           float(st_selection[1].stats.endtime),float(st_selection[2].stats.endtime))
    min_endt = min(mylist2)
    id_m = mylist2.index(min_endt)

    min_endt = UTCDateTime(min_endt)
#    print(max_startt,min_endt)
    st_selection = st_selection.slice(starttime=max_startt,endtime=min_endt)

    timevec = np.linspace(float(st_selection[0].stats.starttime),
                          float(st_selection[0].stats.endtime),st_selection[0].stats.npts)

    search_room_N = st_selection[0].data[np.where((timevec>float(t)+t_SKS-twin) & (timevec<float(t)+t_SKS+twin))]    
    search_room_E = st_selection[1].data[np.where((timevec>float(t)+t_SKS-twin) & (timevec<float(t)+t_SKS+twin))]     
    #search_room_Z = st_selection[0].data[np.where((timevec>float(t)+t_SKS-twin) & (timevec<float(t)+t_SKS+twin))]        

    max_trans_vec = np.sqrt((search_room_N)**2+(search_room_E)**2)
    max_ampl_N = max(abs(search_room_N))
    max_ampl_E = max(abs(search_room_E))
    #max_ampl_Z = max(abs(search_room_Z))
    max_trans = max(max_trans_vec)

    max_ampl = max_trans 

    id_x = np.where(max_trans_vec==max_ampl)
    ## theoretical arrival
#    print 'theoretical SKS ',UTCDateTime(float(t)+t_SKS)
    ## pick the real arrival time
    t_SKS_real = timevec[np.where((timevec>float(t)+t_SKS-twin) & (timevec<float(t)+t_SKS+twin))][id_x]

    
    #### CALC Signal to Noise ratio from max amplitude and from absolute average of trace 15 seconds outside the signal
    S = max_ampl
    secs= 15     ## like 15 s outside of max
    N1= np.mean(abs(st_selection[0].data[id_x[0][0]-10*secs*2:id_x[0][0]-10*secs]))
    N2 = np.mean(abs(st_selection[0].data[id_x[0][0]+10*secs:id_x[0][0]+10*secs*2]))
    N = np.mean([N1,N2])
    SNR = S/N
    
    if np.isnan(SNR)==True:
        SNR=0
        
        
    abs_diff = abs(float(t_SKS_real)-float(t)-t_SKS)    
    st_cut = st_selection.slice(starttime=UTCDateTime(t_SKS_real)-twin,
                                 endtime=UTCDateTime(t_SKS_real)+twin,nearest_sample=True)
    
    ### ONLY PLOTTING BELOW    
    if Splitting_windows==True:    
        fig = plt.figure(figsize=(16,9))
        ax = fig.add_axes([0.15,0.15,0.7,0.7])    

        ax.plot(timevec[0:36000], st_selection[0].data[0:36000], "b-",label=st_selection[0].stats.channel)
        ax.plot(timevec[0:36000], st_selection[1].data[0:36000], "g-",label=st_selection[1].stats.channel)
        ax.plot(timevec[0:36000], st_selection[2].data[0:36000], "k-",label=st_selection[2].stats.channel, linewidth=0.75, alpha=0.5)

        ax.vlines(float(t)+t_SKS, min(st_selection[0].data), max(st_selection[0].data), color='r', linewidth=2,label='SKS-arrival (tau-p)')
        ax.vlines(t_SKS_real,-max_ampl,max_ampl,color='red',linestyle='dashed', linewidth=2,label='SKS-arrival (picked)')
        ax.vlines(float(t)+t_SKKS,min(st_selection[0].data),max(st_selection[0].data),color='chocolate', linewidth=1,label='SKKS-arrival (tau-p)')
        ax.set_title('SKS-window, station: {0} , SNR={1}'.format(st_selection[0].stats.station,int(SNR)),loc='left')

        if float(t_SKS_real)-float(t)>t_SKS:
            ax.text(t_SKS_real-abs_diff/2-1,1.05*-max_ampl,r'$\Delta $'+'t=+{0:.2f} s'.format(abs_diff),fontweight='bold')
        else:
            ax.text(t_SKS_real+abs_diff/2-1,1.05*-max_ampl,r'$\Delta $'+'t=-{0:.2f} s'.format(abs_diff),fontweight='bold')        

        ax.text(float(t)+t_SKS-twin+1,max_ampl*0.95,'Earthquake: \n t={0}, dist={1}$^\circ$, Mw={2}'.format(ev_time.strftime('%Y-%m-%d, %H:%M:%S'),round(ev_dist,2),ev_mag),fontweight='bold')
    ########### SET PROPER TIME AXIS    
        xxticks = np.linspace(timevec[0], timevec[-1],300)
        xxlabels=[]
        for i in range(0,len(xxticks)):
            tmp=UTCDateTime(xxticks[i]).strftime('%H:%M:%S')
            xxlabels.append(tmp)
    ########### SET PROPER TIME AXIS    
        ax.set_xticks(xxticks)
        ax.set_xticklabels(xxlabels)    

        ax.set_xlim(float(t)+t_SKS-twin,float(t)+t_SKS+twin)
        ax.set_ylim(-max_ampl*1.25,max_ampl*1.25)    
        ax.fill_between([float(t_SKS_real)-twin, float(t_SKS_real)+twin], [-max_ampl*1.25, -max_ampl*1.25],
                                 [max_ampl*1.25,max_ampl*1.25],
                                 color='gray', alpha=0.25, label='selected time window $\pm$'+'{0} s'.format(twin))
        ax.set_xlabel('Time [hh:mm:ss]')
        ax.set_ylabel('displacement [m]')    
        ax.grid(alpha=0.5)
        ax.legend(loc=4, bbox_to_anchor=(1.15, 0.8))
        ### SAVE EVERY IMAGE_SOMEWHERE
        try:  
            path_SWindows='{0}/../SplitWave_Results/Splitting_windows/{1}/'.format(save_loc,st_cut[0].stats.station)
            os.mkdir(path_SWindows)
        except:
            pass

        image_name='{0}/../SplitWave_Results/Splitting_windows/{1}/{1}.{2}.png'.format(save_loc,st_cut[0].stats.station,str(t))        
        fig.savefig(image_name)
        plt.close()
            
    return st_cut,SNR,t_SKS_real

    ## save images, return sliced_traces, time_windows
    ## calc signal to noise mean amplitude outside max pick, 

    
## Add EQ
## Add better Time Axis
## add S,SKS,PP Phases
## calc max only on horizontal components
## plot epicentral distances



# Function to Save measurements


In [10]:
def write_head(path, method, station, header):
    fnam_out = os.path.join(path, '{1}/SKS_Splitting_{0}_{1}.txt'.format(station,method))
    print('wrote into:', fnam_out)
                            
    with open(fnam_out, 'w') as fid:
        fid.write('# ' + header + '\n')
        
def write_head_CHEV(path, method, header):
    fnam_out = os.path.join(path, '{0}/SKS_Splitting_{0}.txt'.format(method))
    print('wrote into:', fnam_out)
                            
    with open(fnam_out, 'w') as fid:
        fid.write('# ' + header + '\n')        
        
                            
def write_SKS_Results(path, method, vals, station, header):
    fnam_out = os.path.join(path,
                            '{1}/SKS_Splitting_{0}_{1}.txt'.format(station,method))
    fmt = '%s \n'    
    vals_array = np.asarray(vals)

    with open(fnam_out, 'a') as fid:      
        arraystring = np.array2string(vals_array[:], 
                                      precision=6,
                                      max_line_width=255)
        fid.write(fmt % (arraystring[1:-1]))

def write_SKS_Results_CHEV(path, method, vals, header):
    fnam_out = os.path.join(path,
                            '{0}/SKS_Splitting_{0}.txt'.format(method))
#    print(fnam_out)
    fmt = '%s \n'    
    vals_array = np.asarray(vals)
#    print(vals_array)

    with open(fnam_out, 'a') as fid:      
        arraystring = np.array2string(vals_array[:], 
                                      precision=4,
                                      max_line_width=255)
        fid.write(fmt % (arraystring[1:-1]))
        
path = '/media/hein/home2/SplitWave_Results/SKS/'

header='Station.code, ' + \
        'Station.lat [°],' + \
        'Station.lon [°],' + \
        'Event time, ' + \
        'Event depth [km], ' + \
        'Event mag, '+ \
        'Event lat [°],' + \
        'Event lon [°],' + \
        'fast dir [°],' + \
        'dfast dir [°],' +\
        'lag [s],' + \
        'dlag [s],' + \
        'SNR dB' 

header2='Station.code, ' + \
        'dt [s],' + \
        'dlag [s],' + \
        'fast dir [°]' + \
        'dfast dir [°], '



# SPLITWAVE ROUTINES
## Set The Time Windows t1 and t2
## choose the lag

In [11]:
import splitwavepy as sw

### Just Visualizing
def Splitwave_Eig3D(st_cut,plot=False):

    tmp = st_cut
    delta = tmp[0].stats.delta
#    print(delta)
    t = sw.Trio(tmp[1].data,tmp[0].data, tmp[2].data,delta=delta)
    #t = sw.Pair(st_slice[1].data,)

    ## write something for time window
    t1 = 10
    t2 = 70
    t.set_window(t1,t2)
#    t.plot()

    b = sw.Eig3dM(t,lags=(3,))
    
    try:  
        path_Methods='{0}/../SplitWave_Results/Methods/Eig3D/{1}/'.format(save_loc,st_cut[0].stats.station)
        os.mkdir(path_Methods)
    except:
        pass
    b.save('/media/hein/home2/SplitWave_Results/Methods/Eig3D/{0}/{0}_{1}.eig'.format(st_cut[0].stats.station,st_cut[0].stats.starttime.strftime("%Y-%m-%d")))    
    if plot==True:
        b.plot()
    

    return b.fast, b.dfast,round(b.lag,4),round(b.dlag,4)


def Splitwave_EigenM(st_cut,plot=False):
    # get data into Pair object and plot
    tmp = st_cut    
    north = tmp[1].data
    east = tmp[0].data
    sample_interval = tmp[0].stats.delta
    realdata = sw.Pair(north, east, delta=sample_interval)
    ## write something for time window    
    t1 = 10
    t2 = 50
    realdata.set_window(t1,t2)
   # realdata.plot()
    
    measure = sw.EigenM(realdata)

    if plot==True:
        m.plot()

    
    return measure.fast, measure.dfast,round(m.lag,4),round(m.dlag,4)


def Splitwave_TransM(st_cut,back_az,plot=False):
    tmp = st_cut    
    north = tmp[1].data
    east = tmp[0].data
    sample_interval = tmp[0].stats.delta
    realdata = sw.Pair(north, east, delta=sample_interval)
    ## write something for time window    
    t1 = 10
    t2 = 70
    realdata.set_window(t1,t2)
    #realdata.plot()
    
    m = sw.TransM(realdata, pol=back_az, lags=(2,))
    try:  
        path_Methods='{0}/../SplitWave_Results/Methods/TransM/{1}/'.format(save_loc,st_cut[0].stats.station)
        os.mkdir(path_Methods)
    except:
        pass
    
    m.save('/media/hein/home2/SplitWave_Results/Methods/TransM/{0}/{0}_{1}.eig'.format(st_cut[0].stats.station,st_cut[0].stats.starttime.strftime("%Y-%m-%d")))    
    
    if plot==True:
        m.plot()

    return m.fast, m.dfast,round(m.lag,4),round(m.dlag,4)


def Splitwave_CrossC(st_cut,plot=False):
    tmp = st_cut    
    north = tmp[1].data
    east = tmp[0].data
    sample_interval = tmp[0].stats.delta
    realdata = sw.Pair(north, east, delta=sample_interval)
    ## write something for time window    
    t1 = 10
    t2 = 70
    realdata.set_window(t1,t2)
  #  realdata.plot()

    m = sw.CrossM(realdata, lags=(2,))
    try:  
        path_Methods='{0}/../SplitWave_Results/Methods/CrossC/{1}/'.format(save_loc,st_cut[0].stats.station)
        os.mkdir(path_Methods)
    except:  
        pass
        
    m.save('/media/hein/home2/SplitWave_Results/Methods/CrossC/{0}/{0}_{1}.eig'.format(st_cut[0].stats.station, st_cut[0].stats.starttime.strftime("%Y-%m-%d")))
    if plot==True:
        m.plot()

    
    return m.fast, m.dfast,round(m.lag,4),round(m.dlag,4)




# fast,dfast,lag,dlag =  Splitwave_EigenM(st_cut)
# print(fast,dfast,lag,dlag)
# fast,dfast,lag,dlag =  Splitwave_TransM(st_cut,back_az)
# print(fast,dfast,lag,dlag)
# fast,dfast,lag,dlag =  Splitwave_CrossC(st_cut)
# print(fast,dfast,lag,dlag)


In [12]:
from obspy.signal.rotate import rotate_ne_rt
from obspy.signal.polarization import particle_motion_odr
import matplotlib.gridspec as gridspec
from scipy.optimize import curve_fit


def SKS_Intensity_Chevrot(st_ev,ev_time,t_SKS,back_azimut,plot=True):
#    SV_Az = []
#    Az = []
    st_ev = st_ev.sort()

#    for ev_step in range(0,len(ev_time_l)):    
    ### SORT IT AS ZNE
    st_stream = obspy.Stream()
    tmp = st_ev[2]
    st_stream +=tmp
    tmp = st_ev[1]
    st_stream +=tmp
    tmp = st_ev[0]
    st_stream +=tmp

    gridspec.GridSpec(2,3)

    arrival_time = ev_time+t_SKS

    ### USE CORRECT ARRIVAL TIME
    ### Take small time window around arrival time 
    twin = 15
    st_stream = st_stream.slice(arrival_time-twin,arrival_time+twin,nearest_sample=True)

    limits=np.max([abs(st_stream[2].data),abs(st_stream[2].data)])*2*10**6

    #### CALC THE POLARIZATION OF PARTICLE MOTION
    ## only accept the upper half for Signal
    noise_level=st_stream[0].data**2+st_stream[1].data**2+st_stream[2].data**2
    azimuth, incidence, az_error, in_error = particle_motion_odr(st_stream, noise_thres=np.mean([np.max(noise_level), np.min(noise_level)])+np.std([np.max(noise_level), np.min(noise_level)]))
#    print(az_error)
    ### ROTATE THE SYSTEM FROM NE TO RT
    st_rot_RT = rotate_ne_rt(st_stream[1].data,st_stream[2].data,180+azimuth)    
    
    radial = st_rot_RT[0]
    r_dot = np.diff(radial)/st_stream[1].stats.delta
    radial = radial[0:len(r_dot)]
    transverse=st_rot_RT[1][0:len(r_dot)]    

    r_2 = np.sum(r_dot**2)
    ### NORMALIZE SPLITTING VECTOR
    SV_EQ = -np.sum(2*r_dot*transverse)/r_2

    
    ### EVENT AZIMUT IS BACK-AZIMUT +180
    if back_azimut+180>360:
        Az = back_azimut-180
    else:
        Az=back_azimut+180
    SV_Az = SV_EQ 

    if plot==True:
    ### Only Plotting Below     
        fig = plt.figure(figsize=(16,9))
#        plt.subplot2grid((2,3), (0,0), colspan=2, rowspan=1)        
        ax1 = fig.add_axes([0.1,0.5,0.5,0.3])    
        ax2 = fig.add_axes([0.65,0.5,0.25,0.3])        
        ax3 = fig.add_axes([0.1,0.1,0.5,0.3])     
        ax4 = fig.add_axes([0.65,0.1,0.25,0.3])            
        
        timevec = np.linspace(float(st_stream[0].stats.starttime),float(st_stream[0].stats.endtime),st_stream[0].stats.npts)

        xxticks = np.linspace(timevec[0], timevec[-1],10)
        xxlabels=[]
        for i in range(0,len(xxticks)):
            tmp=UTCDateTime(xxticks[i]).strftime('%H:%M:%S')
            xxlabels.append(tmp)
            
    ########### SET PROPER TIME AXIS    


        ax1.plot(timevec,st_stream[1].data*10**6,'g',label='North')
        ax1.plot(timevec,st_stream[2].data*10**6,'b',label='East')
        ax1.vlines(x=float(arrival_time),ymin=1.3*np.min(np.min([st_stream[1].data*10**6,st_stream[2].data*10**6])),ymax=1.3*np.max(np.max([st_stream[1].data*10**6,st_stream[2].data*10**6])),color='k',linewidth=0.5,label='SKS-Phase')
        ax1.set_title('{0}, SKS-arrival at: {1}, Backazimut={2} $^\circ$, SI={3}'.format(st_stream[0].stats.station,arrival_time.strftime('%Y-%m-%d, %H:%M:%S'),round(Az,2),round(SV_Az,2)))
        ax1.set_xlabel('Time [s]')
        ax1.set_ylabel('displacement [$\mu$m]')  
        ax1.set_xlim(timevec[0],timevec[-1])            
        ax1.set_xticks(xxticks)
        ax1.set_xticklabels(xxlabels)            
        ax1.grid()        
        ax1.legend()

#        plt.subplot2grid((2,3), (0,2))
        ax2.plot(st_stream[2].data*10**6,st_stream[1].data/10**-6,color='black',linestyle='dashed')
        ax2.set_xlabel('East disp. [$\mu$m]')
        ax2.set_ylabel('North disp. [$\mu$m]')
        ax2.axis('equal')
        ax2.set_xlim(-limits, limits)
        ax2.set_ylim(-limits, limits)
        ax2.grid()
        ax2.set_title('Polarization: Azimuth={0}$^\circ$'.format(round(azimuth,2)))
        
#        limits = 1        
#        limits = np.max([np.max(radial),np.max(transverse)])
    
# #        plt.subplot2grid((2,3), (1,0), colspan=2, rowspan=1)
        ax3.plot(timevec[0:-1],radial*10**6,'r',label='Radial')
        ax3.plot(timevec[0:-1],transverse*10**6,'b',label='Transverse') 
        ax3.plot(timevec[0:-1],-0.5*r_dot*(np.max(transverse)/np.max(r_dot))*10**6,color='g',label='radial-derivate',alpha=0.5,linewidth=0.5)
        ax3.vlines(x=float(arrival_time),ymin=1.3*np.min(np.min([st_stream[1].data*10**6,st_stream[2].data*10**6])),ymax=1.3*np.max(np.max([st_stream[1].data*10**6,st_stream[2].data*10**6])),color='k',linewidth=0.5,label='SKS-Phase')        
        ax3.set_xlabel('Time [s]')
        ax3.set_ylabel('displacement [$\mu$m]')                        
        ax3.set_xlim(timevec[0],timevec[-1])            
        ax3.set_xticks(xxticks)
        ax3.set_xticklabels(xxlabels)                    
        ax3.grid()        
        ax3.set_title('rotated System')        
        ax3.legend()

#        plt.subplot2grid((2,3), (1,2))
        ax4.plot(radial*10**6,transverse*10**6,color='black',linestyle='dashed')
        ax4.set_xlabel('Radial disp. [$\mu$m]')
        ax4.set_ylabel('Transverse  disp. [$\mu$m]')
        ax4.axis('equal')        
#        ax4.set_xlim(-limits, limits)
#        ax4.set_ylim(-limits, limits)
        ax4.grid()

        try:  
            path_Methods='/media/hein/home2/SplitWave_Results/Splitting_Intensity/{0}/'.format(st_stream[0].stats.station)
            os.mkdir(path_Methods)
        except:  
            pass
        plt.savefig('/media/hein/home2/SplitWave_Results/Splitting_Intensity/{0}/{0}_{1}'.format(st_stream[0].stats.station,arrival_time.strftime('%Y-%m-%d, %H:%M:%S')))
        plt.close()
#        fig.close()
#    plt.show()

    return Az,SV_Az

## SINOSOID FUNCTION TO BE FIT THROUGH DATA
def func(x, delta_t, phi):
    y = delta_t *np.sin( 2*(np.radians(x)-phi))
    return y


def get_best_dt_and_phi(Az,SV_Az,station, plot=True):
#    print(SV_Az)
    popt = np.zeros(2)
    #try:

    # sort out extrem values
    Az = np.asarray(Az)
    SV_Az = np.asarray(SV_Az)
    xdata = Az
    ydata = SV_Az
    
#    allow_max = 1.3 # lets see how good that works
#    xdata = Az[np.where((SV_Az<=allow_max) & (SV_Az>=-allow_max))]
#    ydata = SV_Az[np.where((SV_Az<=allow_max) & (SV_Az>=-allow_max))]
    
    ### FIT CURVE THROUGH DATA
    popt, pcov = curve_fit(func, xdata, ydata,  bounds=([0, -np.pi], [5, np.pi]))
    azi_theo = np.linspace(0,360)
    perr = np.sqrt(np.diag(pcov))
    if plot==True:
        plt.figure(figsize=(16,8))
        plt.plot(xdata, ydata, 'ko',label='data')

#        plt.plot(xdata, func(xdata, *popt), 'rx',label='prediction')
        
        plt.errorbar(xdata, ydata, yerr=func(xdata, perr[0], perr[1]*180/np.pi)-ydata,color='red',alpha=0.5 ,label='errorbar', fmt='o')
#       plt.plot(azi_theo,func(azi_theo,popt[0]+perr[0],(popt[1]+perr[1])*180/np.pi),color='red',linestyle='dashed',linewidth=0.5,label='1x $\sigma$')
#       plt.plot(azi_theo,func(azi_theo,popt[0]-perr[0],(popt[1]-perr[1])*180/np.pi),color='red',linestyle='dashed',linewidth=0.5)        
        #plt.plot(azi_theo,func(azi_theo,popt[0]+perr[0],(popt[1])*180/np.pi),color='red',linestyle='dashed',linewidth=0.5)    
        plt.plot(azi_theo,func(azi_theo,popt[0],popt[1]),color='black',linestyle='dashed',label='fit')
        plt.xlabel('Azimuth [$^\circ$]')
        plt.ylabel('Amplitude')
        plt.grid()
        plt.xlim(0,360)
#        plt.ylim(-2,2)
#        plt.ylim(-allow_max-1,allow_max+1)        
        plt.title('{4}, best parameters $\Delta$t={0}$\pm${1} s, $\phi$={2}$\pm${3}$^\circ$'.format(round(popt[0],2),round(perr[0],3),round(popt[1]*180/np.pi,2),round(perr[1]*180/np.pi,3),station))
        plt.legend()
        plt.savefig('/media/hein/home2/SplitWave_Results/SKS/Chevrot/Splitting_Intensity_fit_{0}.png'.format(station))
        plt.close()

#     except:
#         print('some Problem')
#         popt[0]=np.nan
#         popt[1]=np.nan
#         pass
    return popt[0],popt[1]*180/np.pi,perr[0],perr[1]*180/np.pi

  
## take time window of N and E wave
## get particle direction for max pulse
## rotate to RT
## calc Splitting Vector 
## generalize for EVENTS
## now calc for several events
## make azimuth vs SV amplitude plot
## fit a Least Square sinusoid

### improve plotting 
### try out the polarization_analysis from obspy
### rotate the result and see if its better constrained?

In [13]:
# from obspy.signal.polarization import polarization_analysis
# stp=50

# # st_ev,st_lat_l,st_lon_l,ev_lat_l,ev_lon_l,ev_time_l,ev_depth_l,ev_mag_l,ev_dist_l,back_azimut_l, t_SKS_l,t_SKKS_l,t_PP_l
# Az,SV_Az,st_stream = SKS_Intensity_Chevrot(st_ev[3*stp:3*stp+3],ev_time_l[stp],t_SKS_l[stp],back_azimut_l[stp],plot=True)

# #print(st_ev[0:3])
# #print(st_stream)
# #st_stream.plot()
# win_len=3
# win_frac = 0.75
# frqlow = 0.01
# frqhigh = 0.5
# stime = st_stream[0].stats.starttime+1
# etime = st_stream[0].stats.endtime-1
# method = 'pm' #'flinn' , 'vidale'
# result_dict =  polarization_analysis(st_stream, win_len, win_frac, frqlow, frqhigh, stime, etime, verbose=False, method='pm', var_noise=0.0)
# #print(result_dict)

# #print(st_stream[0].data** 2 + st_stream[1].data** 2 + st_stream[2].data ** 2)
# noise_level=st_stream[0].data** 2 + st_stream[1].data** 2 + st_stream[2].data ** 2

# plt.plot(noise_level)
# azimuth, incidence, az_error, in_error = particle_motion_odr(st_stream, noise_thres=1*10^-12)
# azimuth, incidence, az_error, in_error = particle_motion_odr(st_stream, noise_thres=np.mean([np.max(noise_level), np.min(noise_level)]))

# #plt.plot(result_dict['timestamp'],result_dict['azimuth'],'r')
# #plt.figure(figsize=(16,9))
# fig, ax1 = plt.subplots()

# # Make the y-axis label, ticks and tick labels match the line color.

# ax2 = ax1.twinx()

# timeve=np.linspace(float(st_stream[0].stats.starttime+1),float(st_stream[0].stats.endtime-1),st_stream[0].stats.npts)
# ax1.errorbar(result_dict['timestamp'],result_dict['azimuth'] , yerr=result_dict['azimuth_error'])
# #print(result_dict['azimuth_error'])
# ax2.plot(timeve,st_stream[1].data,'b')
# ax2.plot(timeve,st_stream[2].data,'g')


## plot the particle motion
#print(np.mean(result_dict['azimuth']))
## weighted mean 
## rotate into Backazimuth of Event angle or from wave polarisation?
### pick stable polarization when maximum of phase is reached
### average over that stable region


#result_dict =  polarization_analysis(st_stream, win_len, win_frac, frqlow, frqhigh, stime, etime, verbose=False, method='flinn', var_noise=0.0)
#plt.plot(result_dict['timestamp'],result_dict['azimuth'],'g')
#result_dict =  polarization_analysis(st_stream, win_len, win_frac, frqlow, frqhigh, stime, etime, verbose=False, method='vidale', var_noise=0.0)
#plt.plot(result_dict['timestamp'],result_dict['azimuth'],'b')


In [14]:
#plt.hist(result_dict['azimuth'],bins=15)
#print(np.mean(result_dict['azimuth']))
#print(new_stat_list[28:])
## find statoin LKBD2

# MAIN PROGRAM 
-> Read in Waveforms -> cut small window around arrival -> run SKS SPlitting

In [15]:
from tqdm import tqdm
import time
import multiprocessing

# def process_ID():
    
    


# for stat in station:

#     multiproc_list = list([[stat,channel[0],day],[stat,channel[1],day],[stat,channel[2],day],[stat,channel[3],day]])
#     p = multiprocessing.Pool(processes=4)
#     p.map(process, multiproc_list)
    
start_time = time.time()
### Choose station from station list

mod = 'ak135' #
#mod = 'iasp91'
model = TauPyModel(model=mod)
save_loc = '/media/hein/home2/SplitWave_Data'
FMT = 'SAC'
Splitting_windows = True
plot_SplitW = False


### Get A list of downloaded Stations
new_stat_list = os.listdir(save_loc)

method= 'Chevrot'
write_head_CHEV(path, method,header2)     

run_MAIN=True
    
if run_MAIN==True:
    
    for item in new_stat_list[25:]:    
    ## all waveforms for the station

        method= 'TransM'
        write_head(path, method, item,header)     
        method= 'CrossC'
        write_head(path, method, item,header)     
        method= 'Eig3D'
        write_head(path, method, item,header)     


        ### Reads in the Station DATE
        
        st_ev,st_lat_l,st_lon_l,ev_lat_l,ev_lon_l,ev_time_l,ev_depth_l,ev_mag_l,ev_dist_l,back_azimut_l, t_SKS_l,t_SKKS_l,t_PP_l = read_station_event_data(item)


        ### Calls Function for Plotting and Splt
        dt,phi,std_dt,std_phi = automatic_SplitWave_Routine(st_ev,st_lat_l,st_lon_l,ev_lat_l,ev_lon_l,ev_time_l,ev_depth_l,ev_mag_l,ev_dist_l,back_azimut_l, t_SKS_l,t_SKKS_l,t_PP_l)

        method='Chevrot'
        ### Still calculate ERROR Bars and improve Quality of method
        write_SKS_Results_CHEV(path, method, [item, dt, std_dt, phi, std_phi], header2)


### go through each single event 
print("--- %s seconds ---" % (time.time() - start_time))

('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/TransM/SKS_Splitting_MUO_TransM.txt')
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/CrossC/SKS_Splitting_MUO_CrossC.txt')
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/Eig3D/SKS_Splitting_MUO_Eig3D.txt')


/home/hein/miniconda3/envs/py27/lib/python2.7/site-packages/obspy/taup/tau_branch.py:496: UserWarning: Resizing a TauP array inplace failed due to the existence of other references to the array, creating a new array. See Obspy #2280.
  warnings.warn(msg)


Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples


  0%|          | 0/87 [00:00<?, ?it/s]/home/hein/miniconda3/envs/py27/lib/python2.7/site-packages/scipy/odr/odrpack.py:860: OdrWarning: Empty data detected for ODR instance. Do not expect any fitting to occur
  OdrWarning)
 38%|███▊      | 33/87 [00:34<00:48,  1.10it/s]/home/hein/miniconda3/envs/py27/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/hein/miniconda3/envs/py27/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 87/87 [01:27<00:00,  1.11it/s]


Splitting intensity analyis: MUO
('dt', 0.56043913416161162)
('phi', 43.389947132028368)
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/TransM/SKS_Splitting_SULZ_TransM.txt')
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/CrossC/SKS_Splitting_SULZ_CrossC.txt')
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/Eig3D/SKS_Splitting_SULZ_Eig3D.txt')
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples


...

Splitting intensity analyis: SENIN
('dt', 1.0594875632829293)
('phi', 37.292116625134533)
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/TransM/SKS_Splitting_ZUR_TransM.txt')
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/CrossC/SKS_Splitting_ZUR_CrossC.txt')
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/Eig3D/SKS_Splitting_ZUR_Eig3D.txt')
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples


100%|██████████| 88/88 [01:27<00:00,  1.03it/s]


Splitting intensity analyis: ZUR
('dt', 0.52245876840141492)
('phi', 51.613406026963588)
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/TransM/SKS_Splitting_LIENZ_TransM.txt')
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/CrossC/SKS_Splitting_LIENZ_CrossC.txt')
('wrote into:', '/media/hein/home2/SplitWave_Results/SKS/Eig3D/SKS_Splitting_LIENZ_Eig3D.txt')
Stream has not 3 channels
Stream has too few samples
Stream has too few samples
Stream has too few samples
no matching file
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
Stream has too few samples
no matching file
no matching file
no matching file
no matching file


100%|██████████| 55/55 [00:54<00:00,  1.06it/s]


Splitting intensity analyis: LIENZ
('dt', 0.40746051852932208)
('phi', 57.167219110003046)
--- 1058.99398494 seconds ---


In [19]:

    
    
## check with r_dot and SV_Az
## set to R_dot
## instedad of abs set to only sum

# current Problems
# S/N ratio sometimes nan --> set to zero
## calculation correct?
## 
## CHEVROT ROUTINE
# normalization values sometimes much greater than 1 , threshold at 5?

## Load in The Error Surfaces!

IndentationError: expected an indented block (<ipython-input-19-2d2832166424>, line 19)

# implementing Chevrot Splitting Intensity
# calc Splitting vector by
$$SV = \frac{\dot{R(\phi)} T(\phi)}{||R||^2}$$

# Diagnostics and Evaluation

In [17]:
## Load in the eig matrixes FROM METHODS
## read the station files and 
## call function def split_TransM(input_stream,....)
## call function def split_crossM(input_stream,....)
# call function def split_Eig3D(input_stream,....)
### save values and 
## measurements list 
## stat, coord, eventinfos, for TransM(, fast, dfast, lag, dlag,), for crossCorr(, fast, dfast, lag, dlag,)
## read in and weighted average of directions
## histogram
## implement QC, splitting Intensity, SNR





                            


# Read in Results files
# make Histograms
# Stack and weighted average
# plot on Map


In [18]:
##### MOVE ALL THE PLOTTING DOWN HERE
### to make a map of study area
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import shapely.geometry as sgeom
#from scalebar import scale_bar


## use topography map?
plt.figure(figsize=(16,9))

proj = ccrs.PlateCarree()
ax = plt.axes(projection=proj)
ax.set_extent([5, 11, 45, 48],proj)
#ax.set_extent([0, 45, 0, 90],proj)

places =  cfeature.NaturalEarthFeature('cultural','populated_places','10m',facecolor='black')
land = cfeature.NaturalEarthFeature('physical','land','10m',
            edgecolor='k',facecolor='lightgoldenrodyellow',)
borders = cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land','10m',
            edgecolor='k',facecolor='none')
lakes = cfeature.NaturalEarthFeature('physical','lakes','10m')
rivers = cfeature.NaturalEarthFeature('physical','rivers_europe','10m')
graticules = cfeature.NaturalEarthFeature('physical','graticules_1','10m')
bounding_box = cfeature.NaturalEarthFeature('physical','wgs84_bounding_box','10m',facecolor='none')
physical_building_blocks = cfeature.NaturalEarthFeature('physical','land_ocean_label_points','10m',facecolor='gray')

states_provinces = cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines',scale='50m')

raster_topo = cfeature.NaturalEarthFeature(category='raster', name='natural-earth',scale='50m')
## add Physical Building Blocks – 10m
### Graticules
### Lakes + Reservoirs
### Rivers
### Railroads
### Roads
### urban areas

# Plotting the data

ax.add_feature(land)
ax.add_feature(borders)
ax.add_feature(lakes)
#ax.add_feature(rivers)
ax.add_feature(graticules)
ax.add_feature(bounding_box)
ax.add_feature(physical_building_blocks)
ax.add_feature(places)
ax.add_feature(states_provinces)
#ax.add_feature(raster_topo)
ax.plot(lon_list,lat_list,'^b',transform=ccrs.PlateCarree(),markersize=10)
for i in range(0,len(new_stat_list)):
    ax.annotate(new_stat_list[i],(lon_list[i],lat_list[i]-0.1),transform=ccrs.PlateCarree(),
        ha='center',va='top',weight='bold')
ax.coastlines()
#ax.background_img()
ext = [5, 11, 45, 48]
sub_ax = plt.axes([0.55,0.12,0.25,0.25], projection=proj)
# Add coastlines and background
sub_ax.coastlines()
#sub_ax.background_img()
# Plot box with position of main map
extent_box = sgeom.box(ext[0],ext[2],ext[1],ext[3])

sub_ax.add_geometries([extent_box], proj, color='none',
                      edgecolor='r', linewidth=2)

#scale_bar(ax,(0.75,0.05),10)

### plot EQ location and Great circle path
plt.show()


# plt.plot(st_lon,st_lat,'bx')
# plt.plot(lon_l,lat_l,'rx')
# plt.show()


OSError: [Errno 12] Cannot allocate memory

In [ ]:
##### JUST OLD STUFF

#for sta in st:
#    print sta.stats.station

stationlist[0]
choose_stat = 'LAUCH'
st_selection = st.select(station=choose_stat)


for checkstat in stationlist:
    sta = checkstat
    if choose_stat==sta.code:
        laa = sta.latitude
        loo = sta.longitude
print(laa)
print(loo)
## plot stations against distance
## get theoretical arrival times 
## cut a window for SKS measurement

dist_deg = locations2degrees(event_info.latitude, event_info.longitude,
                           laa,loo)

print(dist_deg)
arrivals = model.get_travel_times(source_depth_in_km=event_info.depth/1000, distance_in_degree=dist_deg)
#print(arrivals)

t_SKS=arrivals[14].time
t_SKKS=arrivals[15].time
print(t_SKS)


#### TO MAKE SURE THE TRACES START AND END AT THE SAME TIME AND HAVE THE SAME amount of samples 
mylist = (float(st_selection[0].stats.starttime),
                       float(st_selection[1].stats.starttime),float(st_selection[2].stats.starttime))
max_startt = max(mylist)
id_m = mylist.index(max_startt)
max_startt = UTCDateTime(max_startt)

mylist2 = (float(st_selection[0].stats.endtime),
                       float(st_selection[1].stats.endtime),float(st_selection[2].stats.endtime))
min_endt = min(mylist2)
id_m = mylist2.index(min_endt)

min_endt = UTCDateTime(min_endt)


twin = 60*2
t= event_info.time

st_selection = st_selection.slice(starttime=max_startt,endtime=min_endt)


timevec = np.linspace(float(st_selection[0].stats.starttime),
                      float(st_selection[0].stats.endtime),st_selection[0].stats.npts)

## Problems with number of NPTS
## negativ and positive excitation!
search_room = st_selection[1].data[np.where((timevec>float(t)+t_SKS-twin) & (timevec<float(t)+t_SKS+twin))]
max_ampl = max(search_room)
id_x = np.where(search_room==max_ampl)

print(max_ampl)
print(id_x)
## theoretical arrival
print(UTCDateTime(t+t_SKS))
## real arrival
t_SKS_real = timevec[np.where((timevec>float(t)+t_SKS-twin) & (timevec<float(t)+t_SKS+twin))][id_x]
print(UTCDateTime(t_SKS_real))


#plt.plot(search_room)
#plt.plot(id_x,max_ampl,'xr')
SNR = max_ampl/(np.mean(np.abs(search_room[10:100])))
print(SNR)


fig = plt.figure()
ax = fig.add_subplot(1,1, 1)
print(st_selection[0].stats.npts)
print(st_selection[1].stats.npts)
print(st_selection[2].stats.npts)
ax.plot(timevec, st_selection[0].data, "b-",label=st_selection[0].stats.channel)
ax.plot(timevec, st_selection[1].data, "g-",label=st_selection[1].stats.channel)
ax.plot(timevec, st_selection[2].data[0:len(st_selection[0].data)], "k-",label=st_selection[2].stats.channel)

ax.vlines(float(t)+t_SKS, min(st_selection[0].data), max(st_selection[0].data), color='r', linewidth=4,label='SKS-arrival (tau-p)')
ax.plot(t_SKS_real,max_ampl,'xr',label='SKS-arrival (picked)')
ax.set_xlim(float(t)+t_SKS-twin,float(t)+t_SKS+twin)



ax.legend()


plt.show()

#st_selection[0].plot()

## investigate small time window around arriving Phase
## cut it out and feed it into Splitwav
## pick maximum amplitude in this window as SKS-arrival


### 2007er Event von den Practicals ausprobieren 

In [ ]:
# Print path ...
#print os.path.join(cartopy.__path__[0], "data", "raster", "natural_earth")
event1.plot()
event_info.depth


In [ ]:
# new_stat_list = os.listdir(save_loc)
# print(len(new_stat_list)) 

# st = obspy.Stream()

# lat_list = np.zeros(len(new_stat_list))
# lon_list = np.zeros(len(new_stat_list))
# count=0

# for item in new_stat_list:
#     for checkstat in stationlist:
#         sta = checkstat
#         if item==sta.code:
#             laa = sta.latitude
#             loo = sta.longitude

#     filepath='{0}/{1}/2014*BH?*'.format(save_loc,item)    
#     ## something which only gets the data for 1 events
#     try:

#         st += read(filepath)   
#         lat_list[count]= laa
#         lon_list[count]= loo
#         count +=1
#     except:
#         print('no matching file')
    
# #sta = stationlist[0]
# lat_list = lat_list[1:]
# lon_list = lon_list[1:]
# plt.plot(lat_list,lon_list,'x')
# plt.show()
# #tr.stats.coordinates

# ## go trough stations and read waveforms
# ## get coordinates of stations
# ## plot against distance to EQ
# print(min(lat_list))
# print(max(lat_list))
# print(min(lon_list))
# print(max(lon_list))
# print(st.select(station='*'))
# print(lat_list)

In [ ]:
# dist_deg = locations2degrees(event_info.latitude, event_info.longitude,
#                            laa,loo)

# print(dist_deg)
# arrivals = model.get_travel_times(source_depth_in_km=event_info.depth/1000, distance_in_degree=dist_deg)
# #print(arrivals)

# t_SKS=arrivals[14].time
# t_SKKS=arrivals[15].time
# print(t_SKS)


# #### TO MAKE SURE THE TRACES START AND END AT THE SAME TIME AND HAVE THE SAME amount of samples 
# mylist = (float(st_selection[0].stats.starttime),
#                        float(st_selection[1].stats.starttime),float(st_selection[2].stats.starttime))
# max_startt = max(mylist)
# id_m = mylist.index(max_startt)
# max_startt = UTCDateTime(max_startt)

# mylist2 = (float(st_selection[0].stats.endtime),
#                        float(st_selection[1].stats.endtime),float(st_selection[2].stats.endtime))
# min_endt = min(mylist2)
# id_m = mylist2.index(min_endt)

# min_endt = UTCDateTime(min_endt)


# twin = 60*2
# t= event_info.time

# st_selection = st_selection.slice(starttime=max_startt,endtime=min_endt)


# timevec = np.linspace(float(st_selection[0].stats.starttime),
#                       float(st_selection[0].stats.endtime),st_selection[0].stats.npts)

# ## Problems with number of NPTS
# ## negativ and positive excitation!
# search_room = st_selection[1].data[np.where((timevec>float(t)+t_SKS-twin) & (timevec<float(t)+t_SKS+twin))]
# max_ampl = max(search_room)
# id_x = np.where(search_room==max_ampl)

# print(max_ampl)
# print(id_x)
# ## theoretical arrival
# print(UTCDateTime(t+t_SKS))
# ## real arrival
# t_SKS_real = timevec[np.where((timevec>float(t)+t_SKS-twin) & (timevec<float(t)+t_SKS+twin))][id_x]
# print(UTCDateTime(t_SKS_real))


# #plt.plot(search_room)
# #plt.plot(id_x,max_ampl,'xr')
# SNR = max_ampl/(np.mean(np.abs(search_room[10:100])))
# print(SNR)


# fig = plt.figure()
# ax = fig.add_subplot(1,1, 1)
# print(st_selection[0].stats.npts)
# print(st_selection[1].stats.npts)
# print(st_selection[2].stats.npts)
# ax.plot(timevec, st_selection[0].data, "k-",label=st_selection[0].stats.channel)
# ax.plot(timevec, st_selection[1].data, "b-",label=st_selection[1].stats.channel)
# ax.plot(timevec, st_selection[2].data[0:len(st_selection[0].data)], "g-",label=st_selection[2].stats.channel)

# ax.vlines(float(t)+t_SKS, min(st_selection[0].data), max(st_selection[0].data), color='r', linewidth=4,label='SKS-arrival (tau-p)')
# ax.plot(t_SKS_real,max_ampl,'xr',label='SKS-arrival (picked)')
# ax.set_xlim(float(t)+t_SKS-twin,float(t)+t_SKS+twin)



# ax.legend()


# plt.show()